## 2. feladat

Térképen jelöljük ki a 29-es busz útvonalát!
Az adatokat SQLite adatbázisból nyerjük ki!

Adatbázis-kezelő: https://sqlitebrowser.org/

Az adatok CSV formátumban letölthetőek innen: https://bkk.hu/apps/gtfs/

Új tábla hozzáadása: File -> Import -> Table from CSV file

## Hint

Az SQLite adatbázishoz így kapcsolódunk, és nyerünk ki nekünk fontos adatokat a routes táblából:

In [1]:
import sqlite3
sqlite_file = 'bkk.db'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
c

a cursor objektum szolgáltatja nekünk a kapcsolatot a python kódunk és az adatbázis között

SELECT utasításokon keresztül nyerhetünk ki az adatbázisból adatokat, itt pl. egy konkrét sort

In [2]:
res = c.execute('SELECT * FROM routes WHERE route_short_name="29"')
res.description

(('agency_id', None, None, None, None, None, None),
 ('route_id', None, None, None, None, None, None),
 ('route_short_name', None, None, None, None, None, None),
 ('route_long_name', None, None, None, None, None, None),
 ('route_type', None, None, None, None, None, None),
 ('route_desc', None, None, None, None, None, None),
 ('route_color', None, None, None, None, None, None),
 ('route_text_color', None, None, None, None, None, None))

Egy SELECT utasítás eredménye sok mindent tartalmaz, például az adatbázis-tábla eredményhez tartozó oszlopainak neveit( a * miatt kapjuk meg mindet )

... tuple típusú, ezért át kell alakítanunk listává ...

In [3]:
names = [tuple[0] for tuple in res.description]
names

['agency_id',
 'route_id',
 'route_short_name',
 'route_long_name',
 'route_type',
 'route_desc',
 'route_color',
 'route_text_color']

Az adatbázis kurzora ilyenkor már a SELECT eredményének első elemén áll. Kiolvasunk egyet ...

In [4]:
row = c.fetchone()
row

(u'BKK',
 u'0290',
 u'29',
 u'',
 u'3',
 u'Szentl\xe9lek t\xe9r H / H\u0171v\xf6sv\xf6lgy',
 u'009FE3',
 u'FFFFFF')

... ami szintén tuple típusú adat, de dolgozhatunk vele listaként is.

A zip nevű függvény képes arra, hogy összefűzze a nevek listáját az értékek listájával...

In [5]:
zipped = zip(names,row)
zipped

[('agency_id', u'BKK'),
 ('route_id', u'0290'),
 ('route_short_name', u'29'),
 ('route_long_name', u''),
 ('route_type', u'3'),
 ('route_desc', u'Szentl\xe9lek t\xe9r H / H\u0171v\xf6sv\xf6lgy'),
 ('route_color', u'009FE3'),
 ('route_text_color', u'FFFFFF')]

... így kapunk egy tuple-kből álló listát.

A dict függvény a tuple-ök listájából dictionary-t csinál, vagyis név-érték párok tárolóját.

In [6]:
route = dict(zipped)
route

{'agency_id': u'BKK',
 'route_color': u'009FE3',
 'route_desc': u'Szentl\xe9lek t\xe9r H / H\u0171v\xf6sv\xf6lgy',
 'route_id': u'0290',
 'route_long_name': u'',
 'route_short_name': u'29',
 'route_text_color': u'FFFFFF',
 'route_type': u'3'}

és most egyben az öt előző lépés:

In [7]:
res = c.execute('SELECT * FROM routes WHERE route_short_name="29"')
names = [tuple[0] for tuple in res.description]
row = c.fetchone()
zipped = zip(names,row)
route = dict(zipped)
route

{'agency_id': u'BKK',
 'route_color': u'009FE3',
 'route_desc': u'Szentl\xe9lek t\xe9r H / H\u0171v\xf6sv\xf6lgy',
 'route_id': u'0290',
 'route_long_name': u'',
 'route_short_name': u'29',
 'route_text_color': u'FFFFFF',
 'route_type': u'3'}

## Output

Ha mindent jól csináltatok, a bus_stops tartalmazza az állomások koordinátáinak listáját, valahogy így:

In [8]:
bus_stops = [
    {'location': (47.542635, 18.964753)},
    {'location': (47.536127, 18.970568)},
    {'location': (47.534574, 18.973997)},
    {'location': (47.530852, 18.978252)},
    {'location': (47.528371, 18.980938)},
    {'location': (47.525837, 18.984168)},
    {'location': (47.52226, 18.990825)},
    {'location': (47.523548, 18.993187)},
    {'location': (47.528352, 18.996317)},
    {'location': (47.528471, 18.999995)},
    {'location': (47.529117, 19.003624)},
    {'location': (47.529779, 19.009078)},
    {'location': (47.529393, 19.011163)},
    {'location': (47.529311, 19.017049)},
    {'location': (47.52855, 19.021629)},
    {'location': (47.528721, 19.024165)},
    {'location': (47.528227, 19.0271)},
    {'location': (47.527562, 19.028415)},
    {'location': (47.527131, 19.031417)},
    {'location': (47.528039, 19.033422)},
    {'location': (47.52759, 19.036729)},
    {'location': (47.528586, 19.039001)},
    {'location': (47.531833, 19.040023)},
    {'location': (47.535152, 19.039969)},
    {'location': (47.538723, 19.0405)},
    {'location': (47.539919, 19.042014)},
    {'location': (47.539095, 19.045373)},
    {'location': (47.539949, 19.046781)}
]

## Ábrázolás a térképen

gmaps nem működött megbízhatóan, 
vmi más megoldást kellett keresni:

https://ipyleaflet.readthedocs.io/en/latest/

Telepítés:

```
pip install ipyleaflet
jupyter nbextension enable --py --sys-prefix ipyleaflet
```

In [9]:
from ipyleaflet import Marker
from ipyleaflet import Map

m = Map(center=(47.51, 19), zoom=12)
for bus_stop in bus_stops:
    marker = Marker(location=bus_stop["location"], draggable=False)
    m.add_layer(marker);

m

A Jupyter Widget

## Fun fact

A True nem False soha

In [10]:
if True == False:
    print("Valami nincs rendben ezzel a világgal :(")
else:
    print("Minden OK, gond egy szál se :)")

Minden OK, gond egy szál se :)


De ha nagyon akarjuk, elronthatjuk:

In [11]:
True = False

És akkor megszakad a tér idő kontinuum:

In [12]:
if True == False:
    print("Valami nincs rendben ezzel a világgal :(")
else:
    print("Minden OK, gond egy szál se :)")

Valami nincs rendben ezzel a világgal :(


... mert a True most már egy változó, aminek False az értéke.

Szerencsére az univerzum rendje mindig helyre állítható valahogy:

In [13]:
True = (42 == 42)

In [14]:
if True == False:
    print("Valami nincs rendben ezzel a világgal :(")
else:
    print("Minden OK, gond egy szál se :)")

Minden OK, gond egy szál se :)


Bővebben itt olvashattok ilyeneket:

https://github.com/satwikkansal/wtfpython

Nem elriasztani akartalak a pythontól, hanem szeretném, ha jobban megértenétek a belső működését.